In [4]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import copy
import keras
from collections import Counter
from node2vec import Node2Vec
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch
from stellargraph.layer import link_classification, GraphSAGE, GCN
from stellargraph.data import BiasedRandomWalk, EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import FullBatchNodeGenerator, Node2VecLinkGenerator, Node2VecNodeGenerator, GraphSAGENodeGenerator, GraphSAGELinkGenerator

# Задание - Предсказание уровня экспресси белка

<img src='https://www.researchgate.net/publication/313504607/figure/fig3/AS:459880453677066@1486655453033/Protein-protein-interaction-PPI-network-of-DEGs-by-STRING-The-interaction-score-was.png'>




<div class="alert alert-info">
<b>Про биологию</b>
    
Экспрессия — процесс, в ходе которого наследственная информация от гена (последовательности нуклеотидов ДНК) преобразуется в функциональный продукт — белок. Уровнем экспрессии называют - количество белка, производящегося в этом процессе. Чем выше экспрессия белка, тем большее количество этого белка появляется в клетках человека. 
    
    

<div class="alert alert-info">    
<b>Важность задачи</b>
    
Существует множество причин необходимости в знании уровня экспресии белка. Например - это позволяет ученым разрабатывать лекарственные средства и оптимизировать их разработку. Теперь вам предстоит побыть в роли биоинформатика и помочь науке!
    
</div>


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой граф взаимойдествия белков. Где узлы это белки, взаимодействие между белками это ребро. 

Для каждого белка известен уровень его экспрессии. Ниже приведен список ребер `edges`. Информация по экспрессии белков, разбитая на `train` и `test`.
   
    
</div>

In [15]:
# Список ребер графа 

edges = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/edges.csv", sep=",") # Подгрузим данные
edges.head()

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986


In [16]:
# Подгрузим тренирочную выборку
train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/train.csv", sep=",") # Подгрузим данные
print(train.shape)
train.head()

(8000, 2)


,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721


In [17]:
# Подгрузим отложенную выборку для валидации
test = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/test.csv", sep=",")
print(test.shape)
test.head()

(2000, 2)


,target,node
0,0.279231,817
1,0.380795,9574
2,0.686527,1607
3,0.303594,4782
4,0.367374,24125


<div class="alert alert-info">
<b>Про Задачу</b>
    
Вам предлагается предсказать экспрессию белков (`target`) по приведенным данным для отложенной выборки. Ответы в отложенной выборке `test` даны вам для самостоятельной валидации.


    
   
    

<div class="alert alert-info">
<b>Замечание и комментарии</b>
    
    

По ряду причин датасет был упрощен так, чтобы выполнялись следующие условия:
* у графа одна компонента связанности. 
* удалены слишком крупные хабы
* плотность связей графа уменьшена
* решить задачу можно классическими ML подходами
    
   

<div class="alert alert-info">
<b>Оценка результатов</b>
    


Оценка точности модели будет оцениваться по метрике MSE на отложенной выборке `test`
        
</div>

<div class="alert alert-info">
<b>Автор задачи</b>

По всем дополнительным вопросами писать Александру Миленькину
* Телеграмм: Alerin75infskin
* Почта: milenkin.aa@phystech.edu
        
</div>

## prepairing and analysis

In [18]:
edges_gnx = nx.from_pandas_edgelist(edges, "node_1", "node_2", create_using=nx.Graph())
print(f"node: {len(edges_gnx.nodes)}, edges: {len(edges_gnx.edges)}")

node: 10000, edges: 594174


In [ ]:
train['target'].describe()

In [ ]:
df = edges.copy()
df['weight'] = 1
df = df.pivot(index='node_1', columns='node_2', values='weight').fillna(0)

In [ ]:
dict_stats = {}
for node in df.index:
    dict_stats[node] = df.loc[node].sum()  # number of edges for each node

In [ ]:
# mean of edges
pd.Series(dict_stats.values()).describe()

# print(f"mean : {int(sum(dict_stats.values()) / len(dict_stats.values()))}")
# print(f"min : {min(dict_stats.values())}")
# print(f"max : {max(dict_stats.values())}")

In [13]:
# nx.draw_networkx(edges_gnx)  # it doesn't make sense

In [14]:
%%time
degree_centrality = nx.degree_centrality(edges_gnx)

CPU times: user 7.49 ms, sys: 0 ns, total: 7.49 ms
Wall time: 7.42 ms


In [26]:
%%time
closeness_centrality = nx.closeness_centrality(edges_gnx)

CPU times: user 13min 37s, sys: 111 ms, total: 13min 37s
Wall time: 13min 37s


In [27]:
%%time
betweenness_centrality = nx.betweenness_centrality(edges_gnx)

CPU times: user 54min 17s, sys: 380 ms, total: 54min 18s
Wall time: 54min 18s


In [35]:
data_for_lr = pd.read_csv('centralities_data.csv').rename(columns={'Unnamed: 0': 'node'}).set_index('node')

degree_centrality = data_for_lr['degree_centrality']
closeness_centrality = data_for_lr['closeness_centrality']
betweenness_centrality = data_for_lr['betweenness_centrality']

### lr only with centralities

In [32]:
data_for_lr = pd.concat([
    pd.DataFrame(pd.Series(degree_centrality), columns=['degree_centrality']),
    pd.DataFrame(pd.Series(closeness_centrality), columns=['closeness_centrality']),
    pd.DataFrame(pd.Series(betweenness_centrality), columns=['betweenness_centrality'])],
    axis=1)

Train = data_for_lr.merge(train.set_index('node'), how='inner', left_index=True, right_index=True)
Test = data_for_lr.merge(test.set_index('node'), how='inner', left_index=True, right_index=True)

In [36]:
lr_model = LinearRegression()

lr_model.fit(Train[['degree_centrality', 'closeness_centrality', 'betweenness_centrality']], Train['target'])

Test['score'] = lr_model.predict(Test[['degree_centrality', 'closeness_centrality', 'betweenness_centrality']])
mean_squared_error(Test['target'], Test['score'])

0.011354069147677646

### FNN for regresion

In [41]:
class FeedforwardNeuralNetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(FeedforwardNeuralNetModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim) 
        self.act1 = torch.nn.ReLU()
        
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim) 
        self.batchnorm2 = torch.nn.BatchNorm1d(hidden_dim) 
        self.act2 = torch.nn.ReLU()
        
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.act1(out)

        out = self.fc2(out)
        out = self.batchnorm2(out)
        out = self.act2(out)
        
        out = self.fc3(out)
        # out = self.sigmoid(out)
        return out

## get embeddings with random walk

In [37]:
edges_sg = StellarGraph.from_networkx(edges_gnx)

In [38]:
%%time
rw_instance = BiasedRandomWalk(edges_sg)

walks = rw_instance.run(nodes=list(edges_sg.nodes()),
                        length=50,
                        n=20,
                        p=0.4)

CPU times: user 20min 33s, sys: 1.18 s, total: 20min 34s
Wall time: 20min 35s


In [39]:
## shuffle walks
random.shuffle(walks)

In [40]:
%%time
vector_size = 40  # just because it's about median for edges by each node
embeddings_model = Word2Vec(walks,
                            vector_size=vector_size,
                            workers=4)

CPU times: user 1min 34s, sys: 244 ms, total: 1min 34s
Wall time: 26.8 s


In [41]:
embeddings_model.wv[0]

array([ 1.8016675 , -3.0037794 , -2.7069683 ,  1.9127227 , -1.9025455 ,
       -1.5921662 ,  0.41452155, -1.3604666 , -2.9584973 , -0.0200114 ,
        1.3894318 ,  1.2748578 ,  5.2002425 , -3.41285   , -0.10872174,
       -2.4098065 , -1.6019953 ,  3.2309704 , -0.30451548, -0.05676685,
       -2.0679681 , -3.0840073 ,  0.5549966 ,  0.20392409,  1.0632231 ,
        0.45110613,  1.908187  , -1.0702413 ,  0.5472529 ,  0.9929532 ,
       -0.51793694,  1.5407206 , -1.2303212 ,  2.00657   ,  2.104186  ,
       -1.0143015 , -0.4962665 ,  5.110513  , -1.3883709 , -1.474673  ],
      dtype=float32)

In [ ]:
## data
X_train = pd.DataFrame()
y_train = train.set_index('node')

for node in y_train.index:  # the sequence is not broken here
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_train = X_train.append(curr_sr)
    
X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
print(X_train.size)

In [ ]:
## data
X_test = pd.DataFrame()
y_test = test.set_index('node')

for node in y_test.index:  # the sequence is not broken here
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_test = X_test.append(curr_sr)
    
X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
print(X_test.size)

In [57]:
print(X_test.size())

torch.Size([2000, 43])


### ffnn

In [43]:
reg_model = FeedforwardNeuralNetModel(vector_size+3, 2*(vector_size+3), 1)

In [44]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(reg_model.parameters(), lr=0.0001)

In [62]:
reg_model = FeedforwardNeuralNetModel(vector_size+3, 2 * (vector_size+3), 1)

loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(reg_model.parameters(), lr=0.0001)

batch_size = 2000

for epoch in range(1000):  # it's usually tqdm here
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = reg_model.forward(X_batch) 
        
        loss_value = loss(preds, y_batch)
        
        optimizer.zero_grad()
        loss_value.backward()
        
        optimizer.step()
    
    reg_model.eval()
    if epoch % 100 == 0:
        print(f'epoch num {epoch}; loss is {loss_value};')

epoch num 0; loss is 0.7663506865501404;
epoch num 100; loss is 0.6530712246894836;
epoch num 200; loss is 0.5073510408401489;
epoch num 300; loss is 0.38811585307121277;
epoch num 400; loss is 0.3467429578304291;
epoch num 500; loss is 0.46351158618927;
epoch num 600; loss is 0.3678891360759735;
epoch num 700; loss is 0.3344036042690277;
epoch num 800; loss is 0.3755332827568054;
epoch num 900; loss is 0.2690568268299103;


In [63]:
mean_squared_error(y_test, reg_model.forward(X_test).reshape(2000).detach().numpy())

0.6125267

## embeddings with Node2Vec

In [30]:
batch_size = 64
vector_size = 40  # just because it's about median for edges by each node

In [31]:
# %%time
# generator = Node2VecNodeGenerator(edges_sg, batch_size)
# node_to_vec = Node2Vec(emb_size=vector_size, generator)
# node_to_vec.in_out_tensors()

# some mistake here, just on library

In [50]:
%%time
n2v = Node2Vec(edges_gnx, dimensions=vector_size, num_walks=20, walk_length=80, )

Computing transition probabilities:   0%|          | 0/10000 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 20/20 [07:27<00:00, 22.40s/it]


CPU times: user 21min 51s, sys: 8.77 s, total: 22min
Wall time: 21min 54s


In [51]:
%%time
node2vec_model = n2v.fit(window=20, min_count=5, batch_words=10)

CPU times: user 18min 10s, sys: 35.8 s, total: 18min 46s
Wall time: 16min 53s


In [52]:
node2vec_model.wv['11142'].size

40

In [1]:
## data
X_train = pd.DataFrame()
y_train = train.set_index('node')

for node in y_train.index:  # the sequence is not broken here
    curr_sr = pd.Series(node2vec_model.wv[str(node)], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_train = X_train.append(curr_sr)
    
X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

In [2]:
## data
X_test = pd.DataFrame()
y_test = test.set_index('node')

for node in y_test.index:  # the sequence is not broken here
    curr_sr = pd.Series(node2vec_model.wv[str(node)], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_test = X_test.append(curr_sr)
    
X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
print(X_test.size)

In [55]:
reg_model = FeedforwardNeuralNetModel(vector_size+3, 2 * (vector_size+3), 1)

loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(reg_model.parameters(), lr=0.0001)

batch_size = 2000

for epoch in range(1000):  # it's usually tqdm here
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = reg_model.forward(X_batch) 
        
        loss_value = loss(preds, y_batch)
        
        optimizer.zero_grad()
        loss_value.backward()
        
        optimizer.step()
    
    reg_model.eval()
    if epoch % 100 == 0:
        print(f'epoch num {epoch}; loss is {loss_value};')

epoch num 0; loss is 1.3015971183776855;
epoch num 100; loss is 0.7300068140029907;
epoch num 200; loss is 0.6180821657180786;
epoch num 300; loss is 0.43670451641082764;
epoch num 400; loss is 0.4730897545814514;
epoch num 500; loss is 0.5012326836585999;
epoch num 600; loss is 0.4272252917289734;
epoch num 700; loss is 0.4739703834056854;
epoch num 800; loss is 0.3758412301540375;
epoch num 900; loss is 0.41991186141967773;


In [56]:
mean_squared_error(y_test, reg_model.forward(X_test).reshape(2000).detach().numpy())

0.7861319

#### result - lin regression of certainlities is the best

# next doesn't work here, because data don't have features

tried it, just for fun and skills

## GraphSage

In [3]:
X_train = pd.DataFrame()
y_train = train.set_index('node')

X_test = pd.DataFrame()
y_test = test.set_index('node')

for node in y_train.index:  # the sequence is not broken here
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_train = X_train.append(curr_sr)
    
for node in y_test.index:
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_test = X_test.append(curr_sr)
    
X_all = pd.concat([X_train, X_test], axis=0)

In [85]:
print(X_all.shape)

(10000, 43)


In [120]:
%%time
batch_size = 64
num_samples = [20, 20, 20]  # list of neighbours by depth
layer_sizes = [vector_size + 3 , vector_size + 3, vector_size + 3]

graph_nx = copy.deepcopy(edges_gnx)

for node, row in X_all.iterrows():
    graph_nx.nodes[node]["feature"] = row.to_numpy()

edges_sg = StellarGraph.from_networkx(graph_nx, node_features='feature')

unsupervised_samples = UnsupervisedSampler(
    edges_sg,
    nodes=list(edges_sg.nodes()),
    length=80,
    number_of_walks=20
)

generator = GraphSAGENodeGenerator(edges_sg, batch_size, num_samples=num_samples)
node_gen = GraphSAGENodeGenerator(edges_sg, batch_size, num_samples).flow([node for node in list(edges_sg.nodes())])

graphsage = GraphSAGE(layer_sizes=layer_sizes, generator=generator, bias=True)

x_inp, x_out = graphsage.in_out_tensors()

CPU times: user 6.09 s, sys: 3.97 ms, total: 6.09 s
Wall time: 6.09 s


/home/zendro/anaconda3/envs/for_mlops/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [123]:
embedding_model = keras.Model(inputs=x_inp, outputs=x_out)
embedding_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.MSE)

In [124]:
node_embeddings = np.row_stack([embedding_model.predict(node[0], verbose=0) for node in node_gen])
print(node_embeddings.shape)

(10000, 43)


didn't get how i can use it, without features.

##  GCN

In [4]:
X_train = pd.DataFrame()
y_train = train.set_index('node')

X_test = pd.DataFrame()
y_test = test.set_index('node')

for node in y_train.index:  # the sequence is not broken here
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_train = X_train.append(curr_sr)
    
for node in y_test.index:
    curr_sr = pd.Series(embeddings_model.wv[node], name=node)
    curr_sr[len(curr_sr)] = degree_centrality[node]
    curr_sr[len(curr_sr)] = betweenness_centrality[node]
    curr_sr[len(curr_sr)] = closeness_centrality[node]
    X_test = X_test.append(curr_sr)
    
X_all = pd.concat([X_train, X_test], axis=0)

In [110]:
graph_nx = copy.deepcopy(edges_gnx)

for node, row in X_all.iterrows():
    graph_nx.nodes[node]["feature"] = row.to_numpy()
    
edges_sg = StellarGraph.from_networkx(graph_nx, node_features='feature')

generator = FullBatchNodeGenerator(edges_sg, method="gcn")
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"],
    generator=generator, dropout=0.7
)

Using GCN (local pooling) filters...
